In [1]:
import cartopy.crs as ccrs
import sys
sys.path.append('/efs_ecco/ECCO/EMU/emu_userinterface_dir/')
import emu_plot_arg_py as ept
import numpy as np
import matplotlib.pyplot as plt
import lib_python
import plot_adj
from pathlib import Path
import xarray as xr
from os.path import join,expanduser
# identify user's home directory
user_home_dir = expanduser('~')
# import the ECCOv4 py library
sys.path.insert(0,join(user_home_dir,'ECCOv4-py'))
import ecco_v4_py as ecco
import cmocean as cmo
from matplotlib import colors as mcolors
from matplotlib.cm import ScalarMappable
import numpy as np
from pathlib import Path
from pprint import pprint
import requests
import s3fs
import zarr
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import cmocean as cmo
from matplotlib.colors import BoundaryNorm
import matplotlib.ticker as mticker
import cmocean as cmo
from matplotlib.patches import Rectangle
import cartopy.feature as cfeature
## Import the data from 



In [2]:
##1. OPEN THE ADJOINT FILES -------------------------------------------
## 1. Open the grid files
grid_dir     = Path('/home/jovyan/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/')
ds_grid      = xr.open_dataset(grid_dir / 'GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc')
## 2. Insert the adjoint file
globals_dict = ept.emu_plot(run_name="/efs_ecco/sdarma/ECCO/EMU/SOWARM/emu_adj_302_302_3_mask3d.0.0_0.0_-69.0_-61.0_50.0_0.0_1",
                            ctrl_num_list=[1, 2, 3, 4, 5, 6, 7, 8],
                            lag_beg=0, lag_end=1000000);
## 3. Take the variables from inside the adjoint
globals_dict['return_vars'].keys()
tmp_adxx_c   = globals_dict['return_vars']['adxx_tauu']
#tmp_adxx_c = globals_dict['return_vars']['adxx_qnet']
#tmp_adxx_c = globals_dict['return_vars']['adxx_saltflux']


Found file: /efs_ecco/ECCO/EMU/emu_userinterface_dir/emu_env.singularity
EMU Input Files directory: /efs_ecco/ECCO/EMU2/emu_input_dir

Specified directory of EMU run to examine: /efs_ecco/sdarma/ECCO/EMU/SOWARM/emu_adj_302_302_3_mask3d.0.0_0.0_-69.0_-61.0_50.0_0.0_1

Reading /efs_ecco/sdarma/ECCO/EMU/SOWARM/emu_adj_302_302_3_mask3d.0.0_0.0_-69.0_-61.0_50.0_0.0_1

Reading Adjoint Tool output ... 

Found file: adxx_empmr.0000000129.data

*********************************************
Read adjoint gradient for empmr
   adxx: adjoint gradient as a function of space and lag
from file /efs_ecco/sdarma/ECCO/EMU/SOWARM/emu_adj_302_302_3_mask3d.0.0_0.0_-69.0_-61.0_50.0_0.0_1/output/adxx_empmr.0000000129.data
 
Zero lag at (week/record) = 1314
Max  lag at (week/record) = 1201
Specified start and end files #: 0 and 1000000

*********************************************
Outputting adxx_empmr

Found file: adxx_pload.0000000129.data

*********************************************
Read adjoint gradient

In [62]:
##2. open the Polar front files 
NSUBAF = xr.open_dataset('/efs_ecco/sdarma/CTOH_NorthernSubantarcticFront_weekly_1993_2023.nc')
POLARF = xr.open_dataset('/efs_ecco/sdarma/CTOH_PolarFront_weekly_1993_2023.nc')
SUBAF  = xr.open_dataset('/efs_ecco/sdarma/CTOH_SubantarcticFront_weekly_1993_2023.nc')

safn   = NSUBAF['SAF_N']
pf     = POLARF['PF']
saf    = SUBAF['SAF']

# 2) Define new end date: last week of February 2017
end_date  = np.datetime64('2017-02-28')  # or use '2017-02-27' if that's how your weeks align
time_vals = safn.time.values

# 3) Find the last weekly index on or before end_date
valid_ixs = np.where(time_vals <= end_date)[0]
if valid_ixs.size == 0:
    raise ValueError("No time values on or before 2017-02-28 found")
end_ix    = valid_ixs[-1]

# 4) Compute start index for 113 weeks back (114 total weeks)
start_ix  = end_ix - 113
if start_ix < 0:
    raise ValueError("Not enough history before Feb 2017 to go back 113 weeks")

# 5) Slice the datasets accordingly
safn_weekly = safn.isel(time=slice(start_ix, end_ix + 1))
pf_weekly   = pf  .isel(time=slice(start_ix, end_ix + 1))
saf_weekly  = saf .isel(time=slice(start_ix, end_ix + 1))
print("Start date:", safn_weekly.time.values[0])
print("End date:  ", safn_weekly.time.values[-1])
print("Number of weeks:", safn_weekly.sizes['time'])

Start date: 2014-12-29T00:00:00.000000000
End date:   2017-02-27T00:00:00.000000000
Number of weeks: 114


In [66]:
##3. Plot the Qnet in a Stereo projection along with the polar front lines
import cartopy.feature as cfeature
colorm       = plt.get_cmap('RdBu')
for ts in range(8,9):
    tiles    = tmp_adxx[ts]              # Shape: (13, 90, 90)
    XC_tiles = ds_grid.XC.values      # Shape: (13, 90, 90)
    YC_tiles = ds_grid.YC.values
    fig      = plt.figure(figsize=(14, 8))
    ax       = fig.add_subplot(1, 1, 1, projection=ccrs.SouthPolarStereo(central_longitude=0))
    ax.set_extent([-180, 180, -90, -40], crs=ccrs.PlateCarree())# Limit to south of -40 degrees latitude
    ax.coastlines(linewidth=0.5)
    ax.set_title(f'Lag Week {ts} from 2/2017', fontsize=20, pad=12)
    # Set vmin and vmax depending on variable (example: for Qnet)
    vmin, vmax = -1.5e-4, 1.5e-4
    n_levels   = 21
    levels     = np.linspace(vmin, vmax, n_levels)
    norm       = BoundaryNorm(levels, colorm.N, clip=True)
    for t in range(13):  # Loop through tile faces
        mesh          = ax.pcolormesh(
            XC_tiles[t], YC_tiles[t], tiles[t],
            transform = ccrs.PlateCarree(),
            cmap      = colorm,
            norm      = norm,
            rasterized=True)
    # Add white land overlay
    land = cfeature.NaturalEarthFeature('physical', 'land', '50m',edgecolor='grey', facecolor='grey')
    ax.add_feature(land, zorder=10)
    # Add colorbar
    cbar = plt.colorbar(mesh, ax=ax, orientation='horizontal', pad=0.05, aspect=50)
    cbar.set_label('Adjoint Qnet', fontsize=26)
    cbar.ax.tick_params(labelsize=16,rotation=45)
    ## Draw a box around the Southern ocean
    box  = Rectangle((-180, -69), 360, 8, facecolor='none',edgecolor='k',linewidth=2,transform=ccrs.PlateCarree())
    ax.add_patch(box)
    ###---------- PLOT THE FRONTS (vary with ts) -------------------------------
    # Which weekly index corresponds to this ts?
    # ts=0 → most recent week (end_ix), ts=1 → one week earlier, etc.
    week_idx = n_weeks - 1 - ts

    fronts   = [
        ('SAF_N', safn_weekly['longitude'], safn_weekly, 'red'),
        ('PF',    pf_weekly['longitude'],   pf_weekly,   'green'),
        ('SAF',   saf_weekly['longitude'],  saf_weekly,  'blue'),
    ]

    for name, lon_da, lat_da, color in fronts:
        lon   = lon_da.isel(time=week_idx).values
        lat   = lat_da.isel(time=week_idx).values
        valid = ~np.isnan(lat)
        ax.plot(
            lon[valid], lat[valid],
            transform = ccrs.PlateCarree(),
            color = color, linewidth=2, label=f'{name} t−{ts}wk'
        )
    ax.legend(loc='lower left', fontsize=12)
    plt.savefig(f'/home/jovyan/efs_ecco/sdarma/FIGURES/ADJOINT/QNET_ADJOINT_STEREO_LAGWEEK_{ts}_SURFACE_EXP.png')
    plt.close(fig)
